Για μοντέλο tflite σε JAVA

Για την περίπτωση του μοντέλου tensorflow που εκπαιδεύτηκε τα βήματα είναι τα εξής. Πρώτον δημιουργούμε τον converter και ορίζουμε τα 
optimizations,που θα εφαρμόσουμε. Πρόκειται για post trainning optimization το οποίο εφαρμόζει quantization.

In [ ]:
converter = tensorflow.lite.TFLiteConverter.from_keras_model(model=model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_quant_model = converter.convert()
open("model.tflite","wb").write(tflite_quant_model)

Για την περίπτωση του tflite μοντέλου που μετατρέψαμε στο προηγούμενο δημιοργούμε ένα android app για να τρέξει.
Αρχικά δημιουργούμε ένα αντικείμενο interpreter.Πρόκειται για deprecated API.Το πιό πρόσφατο είναι το Model.Στην συνέχεια φορτώνουμε το μοντέλο.

In [ ]:
tflite = new Interpreter(loadModelFile());

private MappedByteBuffer loadModelFile() throws IOException {
        AssetFileDescriptor fileDescriptor=this.getAssets().openFd("model.tflite");
        FileInputStream inputStream=new FileInputStream(fileDescriptor.getFileDescriptor());
        FileChannel fileChannel=inputStream.getChannel();
        long startOffset=fileDescriptor.getStartOffset();
        long declareLength=fileDescriptor.getDeclaredLength();
        return fileChannel.map(FileChannel.MapMode.READ_ONLY,startOffset,declareLength);
    }

Και τέλος δημιουργόυμε τις input/output παραμέτρους και κάνουμε το inference step.Να σημειωθεί οτι πρόκειται για πολύ απλό μοντέλο όσον αφορά το 
input/output.Για πιό πολύπλοκα μοντέλα όπως πχ ένα που έχει είσοδο είκόνα μπορεί να χρησημοποιηθεί η tflite support library που περιέχει utilities που απλοποιούν την διαδικασία.

In [ ]:
private float doInference(String inputString) {
        float[] inputVal=new float[1];
        inputVal[0]=Float.parseFloat(inputString);
        float[][] output=new float[1][1];
        tflite.run(inputVal,output);
        return output[0][0];
    }

Για την περίπτωση deployment με javascript έχουμε τα εξής βήματα. Αρχικά μετατρέπουμε το μοντέλο και εφαρμόζουμε optimizations όπως κάναμε και προηγουμένως.

In [ ]:
tensorflowjs.converters.save_keras_model(model, './')

Στην συνέχεια δημιουργούμε μία σελίδα .html. Η σελίδα αυτή φορτώνει το javascript tensorflow καθώς και ένα script που θα κάνει το inference.Η εφαρμογή χρησημοποιεί WebView για να τρέξει html/javascript. Τα αρχεία json/bin του μοντέλου βρίσκονται ενσωματομένα στο apk και φορτώνονται στην WebView. Για να λειτουργήσει πρέπει να επιτρέψουμε την εκτέλεση javascript καθώς και να ενεργοποιήσουμε την πρόσβαση σε τοπικά αρχεία.Να σημειωθεί ότι αυτή η μέθοδος αφορά μόνο WebView.Σε κανονικό browser δεν λειτουργεί για λόγους ασφαλείας και ο μόνος τρόπος που μπορεί να λειτουργήσει είναι κάνοντας upload τα απαιτούμενα αρχεία μέσω html.

In [ ]:
const model = await tf.loadLayersModel("https://appassets.androidplatform.net/assets/model.json");
y = model.predict(tf.zeros([1]))
document.getElementById("demo").innerHTML = y; 

Για την περίπτωση της pytorch αρχικά μετατρέπουμε το μοντέλο και εφαρμόζουμε optimizations όπως περιγράφηκε στο report

In [ ]:
script_module = torch.jit.script(model)
script_module_optimized = optimize_for_mobile(script_module)
script_module_optimized._save_for_lite_interpreter("model.ptl")

Στην συνέχεια αφού εισάγουμε την βιβλιοθήκη φορτώνουμε το μοντέλο απο τα assets και δημιουργούμε ένα δείγμα εισόδου και παράγουμε την έξοδο.

In [ ]:
Module module = Module.load(assetFilePath(this, "model.ptl"));
float[] a = new float[]{1.0f};
Tensor input = Tensor.fromBlob(a,new long[]{1,1});
Tensor outputTensor = module.forward(IValue.from(input)).toTensor();
float[] scores = outputTensor.getDataAsFloatArray();